# Project 1: Apache Cassandra
---
**There are 2 parts of the project**
1. Part I. ETL Pipeline for Pre-Procesing the Files
2. Part II. Create Table, Insert Data, and Execute the Designed Queries with Apache Cassandra

---
# Part I. ETL Pipeline for Pre-Processing the Files
---

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


After complete execution of part I code, the `event_datafile_new.csv` which is the data for the part II is ready.

---
# Part II. Create Table, Insert Data, and Execute the Designed Queries with Apache Cassandra
---
The `event_datafile_new.csv` file are contain the following columns
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the `event_datafile_new.csv` after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

`Task1` Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


`Task2` Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

`Task3` Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## For each task, There are 3 steps for complete the task.

`step1` Create the query to answer the question

`step2` Implement the code for creating table in Apache Cassandra

`step3` Execute the `step1` and display the result

## `Task1`: Song detail for specified the `session_id` and `item_in_session`
`Detail`: Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

designed table's name: `song_detail_by_session_item`

`step 1`: Create the query to answer the question

In [9]:
query1 = """select artist_name, song_title, song_length \
            from song_detail_by_session_item \
            where session_id = 338 and item_in_session = 4"""

`step 2`: implement the code for creating table based on query in `step 1` then insert data from csv to the table

In [10]:
# create table query
query = "CREATE TABLE IF NOT EXISTS song_detail_by_session_item "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, \
song_length double, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# insert data from event_datafile_new.csv to cassandra
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_detail_by_session_item (session_id, item_in_session, artist_name, \
        song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### `Primary key selection` for `Task1`

- From task's detail, the primary key of session_id and item_in_session should be the primary key in order to achieve the designed query

`step3` Execute the `step1` query and display the result in dataframe

In [11]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

df_result1 = pd.DataFrame(columns=['artist_name', 'song_title', 'song_length'])
for i, row in enumerate(rows):
    df_result1.loc[i] = [row.artist_name, row.song_title, row.song_length]

### The result of designed query for `Task1`

In [12]:
df_result1

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


## `Task2`: Overall details of song and user by user_id and session_id
`Detail`: Provide name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

designed table's name: `overall_detail_by_user_session`

`step 1`: Create the query to answer the question

In [13]:
query2 = """select artist_name, song_title, first_name, last_name \
            from overall_detail_by_user_session \
            where user_id = 10 and session_id = 182"""

`step 2`: implement the code for creating table based on query in `step 1` then insert data from csv to the table

In [14]:
# create table query
query_create_2 = "CREATE TABLE IF NOT EXISTS overall_detail_by_user_session "
query_create_2 = query_create_2 + "(user_id int, session_id int, item_in_session int, artist_name text, \
song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query_create_2)
except Exception as e:
    print(e)
    
# insert data from event_datafile_new.csv to cassandra
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO overall_detail_by_user_session (user_id, session_id, item_in_session, artist_name, \
        song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### `Primary key selection` for `Task2`
1. First requirement: Execute the query with provided information (user_id = 10 and session_id = 182)
    
   - I decided to choose user_id and session_id as partition key for `task2`'s table


2. Second requirement: Sort the artist name by item_in_session

   - I need to add `item_in_session` as cluster key in order to achieve the second requirement

`step3` Execute the `step1` query and display the result in dataframe

In [15]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

df_result2 = pd.DataFrame(columns=['artist_name', 'song_title', 'first_name', 'last_name'])
for i, row in enumerate(rows):
    df_result2.loc[i] = [row.artist_name, row.song_title, row.first_name, row.last_name]

### The result of designed query for `Task2`

In [16]:
df_result2

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## `Task3`: Song detail for specified the `session_id` and `item_in_session`
`Detail`: Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

designed table's name: `users_by_song`

`step 1`: Create the query to answer the question

In [17]:
# create table query
query3 = """select first_name, last_name \
            from users_by_song \
            where song_title = 'All Hands Against His Own'"""

`step 2`: implement the code for creating table based on query in `step 1` then insert data from csv to the table

In [18]:
# create table based on query3
query_create_3 = "CREATE TABLE IF NOT EXISTS users_by_song "
query_create_3 = query_create_3 + "(song_title text, user_id int, \
                first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query_create_3)
except Exception as e:
    print(e)        

# insert data from event_datafile_new.csv to cassandra
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_by_song (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### `Primary key selection` for `Task3`
- In order to acheive the requirement, I decided to choose song_tile and user_id as primary key for `task3`'s table
- Reason: The requirement need only user name who listened to the specified song, even though user listened many times the answer of this question will be the same 

`step3` Execute the `step1` query and display the result in dataframe

In [19]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

df_result3 = pd.DataFrame(columns=['first_name', 'last_name'])
for i, row in enumerate(rows):
    df_result3.loc[i] = [row.first_name, row.last_name]

### The result of designed query for `Task3`

In [20]:
df_result3

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [21]:
drop_table_list = ['song_detail_by_session_item', 'overall_detail_by_user_session', 'users_by_song']
for table_name in drop_table_list:
    query = f"drop table {table_name}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()